<a href="https://colab.research.google.com/github/talhaanwarch/mia-covid19/blob/main/resized_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f81d4715-9bb8-ffe5-01e5-80185e7015bf)


In [ ]:
#connect drive with colab notebook/
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/covid/train.zip /content/
!cp /content/drive/MyDrive/covid/val.zip /content/

In [ ]:
%%capture 
!unzip /content/train.zip
!rm /content/train.zip
!mv /content/resized /content/train


In [ ]:
%%capture 
!unzip /content/val.zip -d val
!rm /content/val.zip


In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import cv2


In [ ]:
# #load data, data loaded is in format of list of list containing folder and image paths
# train_pos_path=[glob(folder+'*.jpg') for folder in glob('train/covid/*/')]
# train_neg_path=[glob(folder+'*.jpg') for folder in glob('train/non-covid/*/')]
# len(train_pos_path),len(train_neg_path)

In [ ]:
# #create labels for list of list images
# train_pos_label=[[0]*len(i)  for i in train_pos_path]
# train_neg_label=[[1]*len(i)  for i in train_neg_path]
# len(train_pos_label),len(train_neg_label)

In [ ]:
# #combine data
# train_path=train_pos_path+train_neg_path
# train_label=train_pos_label+train_neg_label
# len(train_path),len(train_label)

In [ ]:
# #plit data 
# from sklearn.model_selection import train_test_split
# train_path,val_path,train_label,val_label=train_test_split(train_path,train_label,test_size=0.2)

In [ ]:
# #flatten list
# train_path = [item for sublist in train_path for item in sublist]
# val_path = [item for sublist in val_path for item in sublist]
# train_label = [item for sublist in train_label for item in sublist]
# val_label = [item for sublist in val_label for item in sublist]
# len(train_path),len(train_label),len(val_path),len(val_label)

In [ ]:
# train_df=pd.DataFrame(zip(train_path,train_label),columns=['img','label'])
# train_df = train_df.sample(frac=1).reset_index(drop=True)
# train_df.to_csv('train_df.csv',index=False)
# train_df.head()

In [ ]:
# val_df=pd.DataFrame(zip(val_path,val_label),columns=['img','label'])
# val_df = val_df.sample(frac=1).reset_index(drop=True)
# val_df.to_csv('val_df.csv',index=False)
# val_df.head()

In [ ]:
# #calculate mean and standard deviation of random images
# mean,std=[],[]
# for i in train_df.img[0:100]:
#     img=cv2.imread(i,0)/255.0
#     mean.append(np.mean(img)),std.append(np.std(img))
    
# print('mean',np.mean(mean))
# print('std',np.mean(std))

In [ ]:
train_df=pd.read_csv('/content/drive/MyDrive/covid/train_df.csv')
val_df=pd.read_csv('/content/drive/MyDrive/covid/val_df.csv')
train_df.head()

,img,label
0,train/non-covid/ct_scan_790/80.jpg,1
1,train/non-covid/ct_scan_580/36.jpg,1
2,train/covid/ct_scan_11/122.jpg,0
3,train/non-covid/ct_scan_199/280.jpg,1
4,train/non-covid/ct_scan_530/234.jpg,1


In [ ]:
import torch 
from torchvision import transforms 
import torchvision.models as models
import torch.nn as nn

In [ ]:
aug=transforms.Compose([
                        transforms.Resize((224,224)),
                        transforms.Grayscale(),
                        transforms.RandomHorizontalFlip(p=0.5),transforms.RandomVerticalFlip(p=0.5),
                        transforms.RandomRotation(degrees=5),
                        
                        transforms.ToTensor(),
                        transforms.Normalize([0.4, ], [0.3,]),

                        ])

In [ ]:
from PIL import Image
class dfloader(torch.utils.data.Dataset):
    
    def __init__(self,img_id,img_label,transform=None):
        self.img_id = img_id    
        self.img_label=img_label
        self.transform=transform
        
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
          index = index.item()
        img_ind=self.img_id[index]
        label_ind=self.img_label[index]

        img = Image.open(img_ind).convert('L')
      
        if self.transform:
           img=self.transform(img) 
  
        return img ,label_ind
    
    def __len__(self):
        return len(self.img_id)

In [ ]:
import torch
import torch.nn as nn
class Fire1(nn.Module):
   
    def __init__(self, in_channels, squeeze_channels,expand_channels):
        super(Fire1, self).__init__()

        # squeeze 
        self.squeeze = nn.Conv2d(in_channels, squeeze_channels, kernel_size=1)
        self.relu = nn.LeakyReLU()

        # expand
        self.expand_1x1 =nn.Conv2d(squeeze_channels, expand_channels, kernel_size=1)
        self.expand_3x3 =nn.Conv2d(squeeze_channels, expand_channels, kernel_size=3,padding=1)
        

    def forward(self, x):
        x = self.squeeze(x)
        x= self.relu(x)
        x = torch.cat([self.expand_1x1(x),self.expand_3x3(x)], dim=1)
        x = self.relu(x)
        return x
    
    
class MyModelV3(torch.nn.Module):
    
    def __init__(self):
        super(MyModelV3, self).__init__()
        self.net = torch.nn.Sequential(
                 nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=2),
                 nn.MaxPool2d(kernel_size=3,stride=2),
                 nn.LeakyReLU(),
                 nn.BatchNorm2d(32),
                 Fire1(in_channels=32, squeeze_channels=16,expand_channels=32),
                 Fire1(in_channels=64, squeeze_channels=16,expand_channels=64),
                 nn.MaxPool2d(kernel_size=3,stride=2),
                 #nn.BatchNorm2d(128),
                 Fire1(in_channels=128, squeeze_channels=32,expand_channels=96),
                 Fire1(in_channels=192, squeeze_channels=32,expand_channels=128),
                 nn.MaxPool2d(kernel_size=3,stride=2),
                 #nn.BatchNorm2d(256),
                 Fire1(256, 48, 160),
                 Fire1(320, 48, 160),
                 #nn.BatchNorm2d(320),
                 nn.Conv2d(in_channels=320,out_channels=128*2,kernel_size=1,stride=2),
                 nn.LeakyReLU(),
                 nn.Conv2d(in_channels=128*2,out_channels=32,kernel_size=1,stride=1),
                 nn.LeakyReLU(),
                 nn.Conv2d(in_channels=32,out_channels=16,kernel_size=1,stride=1),
                 nn.LeakyReLU(),#just for testing
                 #nn.Conv2d(in_channels=32,out_channels=16,kernel_size=1,stride=1),
                 nn.Flatten(),
                 #PrintShape(),
                 nn.Linear(784, 1),
                 # nn.Linear(512*2, 512)
                
               
                )        
    def forward(self, x):
        return self.net(x)    

In [ ]:
from sklearn.metrics import classification_report,f1_score
from time import time

def get_accuracy(y_true, y_pred):
    assert y_true.ndim == 1 and y_true.size() == y_pred.size()
    y_pred = y_pred > 0.5
    return (y_true == y_pred).sum().item() / y_true.size(0)

def fit_train(loader):
    loss_sum=0
    acc_sum=0
    scaler = torch.cuda.amp.GradScaler() 
    for batch in loader:
        img,label=batch
        with torch.cuda.amp.autocast(): 
          img,label = img.to(device),label.to(device)
          out=model(img)
          loss=criterion(out.view(-1),label.float())
        opt.zero_grad()
        scaler.scale(loss).backward()
        #loss.backward()
        scaler.step(opt)
        #opt.step()
        scaler.update()
        loss_sum+=loss.item()
        acc_sum += get_accuracy(label,out.view(-1))
    return loss_sum,acc_sum
def fit_val(loader):
    loss_sum=0
    acc_sum=0
    for batch in loader:
        img,label=batch
        img,label = img.to(device),label.to(device)
        out=model(img)
        loss=criterion(out.view(-1),label.float())
        loss_sum+=loss.item()
        acc_sum += get_accuracy(label,out.view(-1))
    return loss_sum,acc_sum

def fit_test(loader):
  test_pred=[]
  for img,_ in loader:
    test_pred.append(model(img.to(device)))
  test_pred=torch.cat(test_pred,dim=0)
  y_pred=test_pred.cpu().numpy()
  return y_pred


def fit(model,train_loader,val_loader,test_loader=None,epoch=10,scheduler_step=None,verbose=None):
  
  train_loss_plt=[]
  val_loss_plt=[]
  train_acc_plt=[]
  val_acc_plt=[]
  for ep in range(epoch):
    start=time()
    #start training loop
    train_loss,train_acc=fit_train(train_loader)
    #start validation loop
    model.eval()
    with torch.no_grad():
        val_loss,val_acc=fit_val(val_loader)
    end=np.round((time()-start)/60,2) #time in minute
    model.train()

    #calculate print and append the results for plotting purpose
    val_avg_loss=np.round(val_loss/len(val_loader),2)#val loss of all batches of one epoch
    train_avg_loss=np.round(train_loss/len(train_loader),2)# train loss of all batches of one epoch
    train_avg_acc=np.round(train_acc/len(train_loader),2)#train acc of all batches of one epoch
    val_avg_acc=np.round(val_acc/len(val_loader),2)#val acc of all batches of one epoch
    if scheduler_step:
      scheduler.step(val_avg_loss)
    if verbose:
      print('Epoch {}, time {} min , train acc  {}, train loss {} , val acc is {}, loss is {}, learning rate is {} '.format
            (ep,end,train_avg_acc,train_avg_loss,val_avg_acc,val_avg_loss,opt.param_groups[0]['lr']))
    train_loss_plt.append(train_avg_loss)  #append loss of training data  
    val_loss_plt.append(val_avg_loss)     #append loss of validation data
    train_acc_plt.append(train_avg_acc)  #append acc of training data  
    val_acc_plt.append(val_avg_acc)     #append acc of validation data
 
 #test phase
  if test_loader:
      model.eval()
      with torch.no_grad():
         y_pred=fit_test(test_loader)
  
  return [train_loss_plt,val_loss_plt,train_acc_plt,val_acc_plt],[model,

In [ ]:
from torch.utils.data.dataloader import DataLoader
train=dfloader(train_df.img.values,train_df.label.values,transform=aug)
val=dfloader(val_df.img.values,val_df.label.values,transform=aug)
train_loader = DataLoader(train,shuffle=True,num_workers=0,batch_size=512)
val_loader = DataLoader(val,shuffle=True,num_workers=0,batch_size=512)

In [ ]:
device=torch.device('cuda')
model=MyModelV3().to(device)
criterion = nn.BCEWithLogitsLoss()
opt=torch.optim.AdamW(params=model.parameters(),lr=0.001)

In [ ]:
res,mod=fit(model,train_loader,val_loader,epoch=15,scheduler_step=False,verbose=True)

Epoch 0, time 13.9 min , train acc  0.45, train loss 0.69 , val acc is 0.47, loss is 0.69, learning rate is 0.001 
Epoch 1, time 14.42 min , train acc  0.6, train loss 0.62 , val acc is 0.68, loss is 0.59, learning rate is 0.001 
Epoch 2, time 14.43 min , train acc  0.74, train loss 0.49 , val acc is 0.73, loss is 0.55, learning rate is 0.001 
Epoch 3, time 14.47 min , train acc  0.82, train loss 0.37 , val acc is 0.75, loss is 0.59, learning rate is 0.001 
Epoch 4, time 14.47 min , train acc  0.88, train loss 0.27 , val acc is 0.75, loss is 0.69, learning rate is 0.001 
Epoch 5, time 14.35 min , train acc  0.92, train loss 0.19 , val acc is 0.76, loss is 0.82, learning rate is 0.001 
Epoch 6, time 14.38 min , train acc  0.94, train loss 0.15 , val acc is 0.78, loss is 0.75, learning rate is 0.001 
Epoch 7, time 14.42 min , train acc  0.95, train loss 0.12 , val acc is 0.78, loss is 0.75, learning rate is 0.001 
Epoch 8, time 14.32 min , train acc  0.96, train loss 0.1 , val acc is 0.7

In [ ]:
state = {
        'epoch': 15,
        'state_dict': model.state_dict(),
        'optimizer': opt.state_dict(),
}
savepath=/content/drive/MyDrive/covid/+'0to15.pt'
torch.save(state,savepath)